# Loading packages

In [1]:
import csv
import pandas as pd
import string

import requests
import datetime
from time import sleep, time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

# Get driver

In [2]:
def get_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(executable_path="C:/Users/sadaouih/Desktop/Github Project/Scraping Data/chromedriver/chromedriver.exe",chrome_options=options)
    return driver

# Connexion to the URL

In [3]:
def connect_to_base(browser,page_number):
    
    base_url = 'https://scholar.google.fr/scholar?start=' + str(page_number) + '&q=perinatal++and+PMSI+&hl=fr&as_sdt=0,5'
    connection_attempts = 0
    while connection_attempts < 3: # Nombre d'essai pour se connecter au site
        try:
            browser.get(base_url)
            WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.ID, 'gs_res_ccl')))
            return True
        except Exception as ex:
            connection_attempts += 1
            print("Error connecting to {}".format(base_url))
            print("Attempt #{}".format(connection_attempts))
    return False

In [4]:
browser = get_driver()
connect_to_base(browser,60)

C:\Users\sadaouih\AppData\Local\Continuum\anaconda3\envs\dscience-env\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


True

# Parse HTML link and get database

In [5]:
def parse_html(html):
    
    soup = BeautifulSoup(html,'html.parser')
    blocks = soup.find_all('div',class_="gs_r gs_or gs_scl")
    output_list = []
    
    for link in blocks:
        title = link.h3.get_text()
        autor = link.find('div',class_='gs_a').get_text()     
        url  =  link.find('a', href=True)['href']  
        try:
            desc  = link.find('div',class_='gs_rs').get_text()
        except Exception as ex:
            desc = "NA"
        page_info = {
            'title': title,
            'autor': autor,
            'desc': desc,
            'url': url
        }
        output_list.append(page_info)
    #dataf = pd.DataFrame(output_list)
    return(output_list)

In [6]:
html1 = browser.page_source
first_page = parse_html(html1)

In [7]:
pd.DataFrame(first_page)

,autor,desc,title,url
0,"S Haddad, JF Secourgeon… - … de médecine périn...",… Keywords Fetal heart rate · Physiopathology ...,Implication d'un réseau de périnatalité dans l...,https://rmp.revuesonline.com/articles/lvrmp/ab...
1,"T Rousseau, C Ferdynus, C Quantin, JB Gouyon… ...",… Birth weight. Gestational age. Sex. Curve of...,… des nouveau-nés issus de grossesses uniques ...,http://www.em-consulte.com/en/article/256074
2,"C Ganzhorn, MJ Boiron, MC Brochier… - Journal ...",… Société Française de Médecine Périnatale 88 ...,74 Travail en réseau maternité-pédiatrie,https://www.sciencedirect.com/science/article/...
3,"A Serfaty, G Bréart - Revue de médecine périna...","… de santé périnatale Building Knowledge, Peri...","[CITATION][C] Construction des connaissances, ...",https://link.springer.com/content/pdf/10.1007/...
4,"Y Ville, G Crépin, G Bréart",NA,[CITATION][C] La mortalité périnatale en France,javascript:void(0)
5,"B Blondel - Santé, Société et Solidarité, 2004...",… et extension des informations collectées pou...,[HTML][HTML] Le système d'information périnata...,https://www.persee.fr/doc/oss_1634-8176_2004_n...
6,"C Jobert-Delatoura, AM Prevostb… - … d'Épidémi...",… conclusion.– Ces quelques indicateurs du PMS...,[PDF][PDF] Évaluation de la précarité en SSR à...,https://www.sante-centre.fr/portail_v1/gallery...
7,"R Boulkedid, O Sibony, C Bossu-Salvador… - … d...",NA,[CITATION][C] Système de contrôle de qualité d...,javascript:void(0)
8,"S Pioc, V Merle, H Marini, V Josset… - … d'Épi...",NA,[CITATION][C] Les droits des usagers du systèm...,javascript:void(0)
9,"J Bouaud, B Séroussi, J Gligorov, JP Lotz… - …...",NA,[CITATION][C] Évaluation des limites à la mise...,javascript:void(0)


In [8]:
browser.quit()

# Export Data

In [9]:
# Run process and see if there an error 

In [10]:
def run_process(page_number, browser):
    if connect_to_base(browser, page_number):
        sleep(1)
        html = browser.page_source
        output_list = parse_html(html)
    else:
        print('Error connecting to google scholar')
    return(output_list)

In [11]:
# Create data frame

In [12]:
if __name__ == '__main__':
    # set variables
    start_time = time()
    current_page = 0
    output_timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    df = []
    #page = [10,20,30,40,50]
    while current_page <= 960:
        browser = get_driver()
        print('Scraping page #{}...'.format(current_page))
        df.append(run_process(current_page, browser))
        current_page = current_page + 10
    browser.quit()
    
    end_time = time()
    elapsed_time = end_time - start_time
    print('Elapsed run time: {} seconds'.format(elapsed_time))

C:\Users\sadaouih\AppData\Local\Continuum\anaconda3\envs\dscience-env\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


Scraping page #0...
Scraping page #10...
Scraping page #20...
Scraping page #30...
Scraping page #40...
Scraping page #50...
Scraping page #60...
Scraping page #70...
Scraping page #80...
Scraping page #90...
Scraping page #100...
Scraping page #110...
Scraping page #120...
Scraping page #130...
Scraping page #140...
Scraping page #150...
Scraping page #160...
Scraping page #170...
Scraping page #180...
Scraping page #190...
Scraping page #200...
Scraping page #210...
Scraping page #220...
Scraping page #230...
Scraping page #240...
Scraping page #250...
Scraping page #260...
Scraping page #270...
Scraping page #280...
Scraping page #290...
Scraping page #300...
Scraping page #310...
Scraping page #320...
Scraping page #330...
Scraping page #340...
Scraping page #350...
Scraping page #360...
Scraping page #370...
Scraping page #380...
Scraping page #390...
Scraping page #400...
Scraping page #410...
Scraping page #420...
Scraping page #430...
Scraping page #440...
Scraping page #450...

In [84]:
# The final datafrmae

In [13]:
df_final  = [val for sublist in df for val in sublist]
df_final = pd.DataFrame(df_final)
df_final.shape

In [68]:
# Year variables

In [81]:
# str.split(str.split(df_final['autor'][400],",")[-1],"-")[0].replace(" ", "")
df_final['year'] = df_final['autor'].apply(lambda x : str.split(str.split(x,",")[-1],"-")[0].replace(" ", ""))

letters = list(string.ascii_letters)
df_final['year'] = df_final['year'].apply(lambda x: pd.isna(x) if x[1] in letters else x)

In [83]:
df_final.head()

,autor,desc,title,url,year
0,"B Cornet, JB Gouyon, C Binquet, P Sagot… - … d...",NA,[CITATION][C] Évaluation régionale en périnata...,javascript:void(0),2001
1,"C Quantin, J Cottenet, A Vuagnat, C Prunet… - ...",Résumé Objectif Comparer les données PMSI rela...,Qualité des données périnatales issues du PMSI...,https://masson.fr/article/843939,2014
2,"A Pierron, M Revert, K Goueslard, A Vuagnat… -...",Résumé Position du problème La France est l'un...,Évaluation de la qualité métrologique des donn...,https://www.researchgate.net/profile/Cottenet_...,2015
3,"PH Bréchat, D Castiel, D Carmona, C Brunner… -...",NA,[CITATION][C] Équité et planification: analyse...,http://fulltext.bdsp.ehesp.fr/Sfsp/SantePubliq...,2008
4,"C Cans, J Fauconnier, A Benbassa, F Olive - 20...",… la pertinence de cet item pour l'épidémiolog...,[HTML][HTML] Mortinatalité et interruptions de...,http://www.em-consulte.com/en/article/114204,False


In [84]:
#df_final.to_excel('Articles-perintal+pmsi.xlsx')